<a href="https://www.kaggle.com/code/sherrytp/scrapping-imdb?scriptVersionId=126213809" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import urllib
import json
# Download IMDB's Top 250 data
url = 'http://www.imdb.com/chart/top'
base = "http://www.imdb.com"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

### option 2
# soup = BeautifulSoup(response.content, 'html.parser')

In [10]:
response.content[:1000]

b'\n\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    \n    \n    \n\n    \n    \n    \n\n    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">\n            <style>\n                body#styleguide-v2 {\n                    background: no-repeat fixed center top #000;\n                }\n            </style>\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n        <title>IMDb Top 250 - IMDb</title>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_post_tit

# Example

In [15]:
test = soup.findAll('img',{"width":45,'height':'67'})
len(test)

250

In [2]:
egClass = "posterColumn"

In [16]:
egList = soup.select("td.posterColumn")
egtd = egList[0]

In [19]:
egtd

<td class="posterColumn">
<span data-value="1" name="rk"></span>
<span data-value="9.216819982109422" name="ir"></span>
<span data-value="7.791552E11" name="us"></span>
<span data-value="2010927" name="nv"></span>
<span data-value="-1.7831800178905777" name="ur"></span>
<a href="/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&amp;pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&amp;pf_rd_r=3254HNZSJ0GN9A9N9D6T&amp;pf_rd_s=center-1&amp;pf_rd_t=15506&amp;pf_rd_i=top&amp;ref_=chttp_tt_1"> <img height="67" src="https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY67_CR0,0,45,67_AL_.jpg" width="45"/>
</a> </td>

In [20]:
egtd.find("img").attrs.get("src")

'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY67_CR0,0,45,67_AL_.jpg'

# Get title Column and link to Child content (jump to child link)

In [3]:
TitleColumn = soup.select(".titleColumn > a")

child = TitleColumn[0]

urlChild= child.attrs.get("href")
base + urlChild

In [20]:
urlContent = requests.get(base + urlChild)

soupChild  = BeautifulSoup(urlContent.text,'lxml')

contentChild = soupChild.select('.flatland')

# Get poster url list

In [30]:
# posterList = soup.findAll('td', class_='posterColumn')
posterList = soup.select('.posterColumn img')
posterUrl = list(map(lambda x:x.attrs.get("src"),posterList))

In [49]:
def downloadImage(url,path):
    try:
        urllib.request.urlretrieve(url, path)
    except:
        raise("download meet an issue")
downloadImage(posterUrl[0],'data/poster/eg.jpg')

# Get movie title, author and others

In [6]:
movies = soup.select('td.titleColumn')
links = [a.attrs.get('href') for a in soup.select('td.titleColumn a')]
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]
votes = [b.attrs.get('data-value') for b in soup.select('td.ratingColumn strong')]

imdb = []

# Store each item into dictionary (data), then put those into a list (imdb)
for index in range(0, len(movies)):
    # Seperate movie into: 'place', 'title', 'year'
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    place = movie[:len(str(index))-(len(movie))]
    data = {"movie_title": movie_title,
            "year": year,
            "place": place,
            "star_cast": crew[index],
            "rating": ratings[index],
            "vote": votes[index],
            "link": links[index]}
    imdb.append(data)

for item in imdb:
    print(item['place'], '-', item['movie_title'], '('+item['year']+') -', 'Starring:', item['star_cast'])
 

1 - The Shawshank Redemption (1994) - Starring: Frank Darabont (dir.), Tim Robbins, Morgan Freeman

2 - The Godfather (1972) - Starring: Francis Ford Coppola (dir.), Marlon Brando, Al Pacino

3 - The Godfather: Part II (1974) - Starring: Francis Ford Coppola (dir.), Al Pacino, Robert De Niro

4 - The Dark Knight (2008) - Starring: Christopher Nolan (dir.), Christian Bale, Heath Ledger

5 - 12 Angry Men (1957) - Starring: Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb

6 - Schindler's List (1993) - Starring: Steven Spielberg (dir.), Liam Neeson, Ralph Fiennes

7 - The Lord of the Rings: The Return of the King (2003) - Starring: Peter Jackson (dir.), Elijah Wood, Viggo Mortensen

8 - Pulp Fiction (1994) - Starring: Quentin Tarantino (dir.), John Travolta, Uma Thurman

9 - The Good, the Bad and the Ugly (1966) - Starring: Sergio Leone (dir.), Clint Eastwood, Eli Wallach

1 -  Fight Club (1999) - Starring: David Fincher (dir.), Brad Pitt, Edward Norton

11 - The Lord of the Rings: The Fello